# (4주차) 9월30일
> 파이토치를 이용하여 회귀모형 학습하기 (2) 

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최규빈

---

In [1]:
import torch 
import numpy as np 

### Data

`-` model: $y_i= w_0+w_1 x_i +\epsilon_i = 2.5 + 4x_i +\epsilon_i, \quad i=1,2,\dots,n$ 

`-` model: ${\bf y}={\bf X}{\bf W} +\boldsymbol{\epsilon}$

- ${\bf y}=\begin{bmatrix} y_1 \\ y_2 \\ \dots \\ y_n\end{bmatrix}, \quad {\bf X}=\begin{bmatrix} 1 & x_1 \\ 1 & x_2 \\ \dots \\ 1 & x_n\end{bmatrix}, \quad {\bf W}=\begin{bmatrix} 2.5 \\ 4 \end{bmatrix}, \quad \boldsymbol{\epsilon}= \begin{bmatrix} \epsilon_1 \\ \dots \\ \epsilon_n\end{bmatrix}$

In [2]:
torch.manual_seed(43052)
n=100
ones= torch.ones(n)
x,_ = torch.randn(n).sort()
X = torch.vstack([ones,x]).T
W = torch.tensor([2.5,4])
ϵ = torch.randn(n)*0.5
y = X@W + ϵ
ytrue = X@W

### step1~2 요약 

#### 방법1: 모델을 직접선언 + loss함수도 직접선언 

In [3]:
What1=torch.tensor([-5.0,10.0],requires_grad=True) 
yhat1=X@What1
loss1=torch.mean((y-yhat1)**2) 
loss1

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법2: 모델식을 torch.nn으로 선언 (bias=False) + loss 직접선언 

In [4]:
net2=torch.nn.Linear(in_features=2,out_features=1,bias=False) 
net2.weight.data= torch.tensor([[-5.0,10.0]]) 
yhat2=net2(X) 
loss2=torch.mean((y.reshape(100,1)-yhat2)**2) 
loss2

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법3: 모델식을 torch.nn으로 선언 (bias=True) + loss 직접선언

In [5]:
net3=torch.nn.Linear(in_features=1,out_features=1,bias=True) 
net3.weight.data= torch.tensor([[10.0]])
net3.bias.data= torch.tensor([[-5.0]]) 
yhat3=net3(x.reshape(100,1)) 
loss3=torch.mean((y.reshape(100,1)-yhat3)**2) 
loss3

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법4: 모델식을 직접선언 + loss함수는 torch.nn.MSELoss() 

In [6]:
What4=torch.tensor([-5.0,10.0],requires_grad=True) 
yhat4=X@What4 
lossfn=torch.nn.MSELoss() 
loss4=lossfn(y,yhat4) 
loss4

tensor(85.8769, grad_fn=<MseLossBackward>)

#### 방법5: 모델식을 torch.nn으로 선언 (bias=False) + loss함수는 torch.nn.MSELoss()

In [7]:
net5=torch.nn.Linear(in_features=2,out_features=1,bias=False) 
net5.weight.data= torch.tensor([[-5.0,10.0]]) 
yhat5=net5(X) 
#lossfn=torch.nn.MSELoss() 
loss5=lossfn(y.reshape(100,1),yhat5) 
loss5 

tensor(85.8769, grad_fn=<MseLossBackward>)

#### 방법6: 모델식을 torch.nn으로 선언 (bias=True) + loss함수는 torch.nn.MSELoss()

In [12]:
net6=torch.nn.Linear(in_features=1,out_features=1,bias=True) 
net6.weight.data= torch.tensor([[10.0]])
net6.bias.data= torch.tensor([[-5.0]]) 
yhat6=net6(x.reshape(100,1)) 
loss6=lossfn(y.reshape(100,1),yhat6) 
loss6

tensor(85.8769, grad_fn=<MseLossBackward>)

### step3: derivation  

#### loss1 

In [13]:
loss1.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [14]:
What1.grad.data

tensor([-13.4225,  11.8893])

- 이것이 손계산을 통한 이론적인 미분값과 일치함은 이전시간에 확인하였음. 

#### loss2

In [15]:
loss2.backward()

In [16]:
net2.weight.grad

tensor([[-13.4225,  11.8893]])

#### loss3 

In [17]:
loss3.backward()

In [18]:
net3.bias.grad,net3.weight.grad

(tensor([[-13.4225]]), tensor([[11.8893]]))

#### loss4

In [19]:
loss4.backward()

In [20]:
What4.grad.data

tensor([-13.4225,  11.8893])

#### loss5 

In [21]:
loss5.backward()

In [22]:
net5.weight.grad

tensor([[-13.4225,  11.8893]])

#### loss6

In [23]:
loss6.backward()

In [24]:
net6.bias.grad,net6.weight.grad

(tensor([[-13.4225]]), tensor([[11.8893]]))

### step4: update 

#### loss1 

In [25]:
What1.data ## update 전 

tensor([-5., 10.])

In [26]:
lr=0.1 
What1.data = What1.data - lr*What1.grad.data ## update 후 
What1

tensor([-3.6577,  8.8111], requires_grad=True)

#### loss2 

In [27]:
net2.weight.data 

tensor([[-5., 10.]])

In [35]:
optmz2 = torch.optim.SGD(net2.parameters(),lr=0.1) 

In [36]:
optmz2.step() ## update 

In [37]:
net2.weight.data ## update 후

tensor([[-2.3155,  7.6221]])

#### loss3

In [31]:
net3.bias.data,net3.weight.data

(tensor([[-5.]]), tensor([[10.]]))

In [32]:
optmz3 = torch.optim.SGD(net3.parameters(),lr=0.1) 

In [33]:
optmz3.step()

In [34]:
net3.bias.data,net3.weight.data

(tensor([[-3.6577]]), tensor([[8.8111]]))

In [38]:
list(net3.parameters())

[Parameter containing:
 tensor([[8.8111]], requires_grad=True),
 Parameter containing:
 tensor([[-3.6577]], requires_grad=True)]

#### loss4

In [39]:
What4.data ## update 전 

tensor([-5., 10.])

In [40]:
lr=0.1 
What4.data = What4.data - lr*What4.grad.data ## update 후 
What4

tensor([-3.6577,  8.8111], requires_grad=True)

#### loss5

In [41]:
net5.weight.data 

tensor([[-5., 10.]])

In [42]:
optmz5 = torch.optim.SGD(net5.parameters(),lr=0.1) 

In [43]:
optmz5.step() ## update 

In [44]:
net5.weight.data ## update 후

tensor([[-3.6577,  8.8111]])

#### loss6

In [45]:
net6.bias.data,net6.weight.data

(tensor([[-5.]]), tensor([[10.]]))

In [46]:
optmz6 = torch.optim.SGD(net6.parameters(),lr=0.1) 

In [47]:
optmz6.step()

In [48]:
net6.bias.data,net6.weight.data

(tensor([[-3.6577]]), tensor([[8.8111]]))

### step1~4를 반복하면된다. 

- loss5를 보면

In [60]:
net=torch.nn.Linear(in_features=2,out_features=1,bias=False) ## 모형정의 
optmz=torch.optim.SGD(net.parameters(),lr=0.1)
mseloss=torch.nn.MSELoss() 
for epoc in range(100): 
    # step1: yhat 
    yhat=net(X) ## yhat 계산 
    # step2: loss
    loss=mseloss(y.reshape(100,1),yhat) 
    # step3: derivation 
    loss.backward() 
    # step4: update
    optmz.step()
    optmz.zero_grad() ## 외우세요.. #기울기를 초기화해준다

In [61]:
list(net.parameters())

[Parameter containing:
 tensor([[2.4459, 4.0043]], requires_grad=True)]

### 숙제 

아래를 실행해보고 결과를 관찰하라. 
```python
net=torch.nn.Linear(in_features=2,out_features=1,bias=False) ## 모형정의 
optmz=torch.optim.SGD(net.parameters(),lr=0.1)
mseloss=torch.nn.MSELoss() 
for epoc in range(100): 
    # step1: yhat 
    yhat=net(X) ## yhat 계산 
    # step2: loss
    loss=mseloss(y.reshape(100,1),yhat) 
    # step3: derivation 
    loss.backward() 
    # step4: update
    optmz.step()
```

In [70]:
net=torch.nn.Linear(in_features=2,out_features=1,bias=False) ## 모형정의 
optmz=torch.optim.SGD(net.parameters(),lr=0.1)
mseloss=torch.nn.MSELoss() 
for epoc in range(100): 
    # step1: yhat 
    yhat=net(X) ## yhat 계산 
    # step2: loss
    loss=mseloss(y.reshape(100,1),yhat) 
    # step3: derivation 
    loss.backward() 
    # step4: update
    optmz.step()

In [71]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.4027, -0.7099]], requires_grad=True)]